### PRIMERA PARTE

In [245]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_graphviz, export_text
from scipy.stats import entropy

In [246]:
#Comenzamos creando la tabla inicial
tabla = pd.DataFrame(columns = ['Presion', 'Clase'], index = np.arange(6))
tabla

,Presion,Clase
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN


In [247]:
#Rellenamos la tabla con los valores del ejercicio de teoría
tabla.Presion = [40, 48, 60, 72, 80, 90]
tabla.Clase = ['+', '+', '-', '-', '-', '+']
tabla

,Presion,Clase
0,40,+
1,48,+
2,60,-
3,72,-
4,80,-
5,90,+


In [248]:
#Creamos una lista llena de 0's y 1's segun corresponda para calcular luego la entropía
list = []
for i in range(len(tabla.Clase)):
    if tabla.Clase[i] == '+':
        list.append(1)
    else:
        list.append(0)
        
H = entropy(tabla.Clase.value_counts(normalize=True), base=2)
H

1.0

A continuación, procedemos a calcular la ganancia de información para la primera cota de digitalización (Presión = 54).
Primero, empleamos el metodo digitize para particionar un conjunto de datos, en este caso, los correspondientes a 
tabla.Presion y obtener los índices del intervalo al que pertece cada valor. Para ello, utilizamos bins = (48+60)/2. 
Si el dato es menor que el valor de bins el índice será 0, por el contrario, si el dato es superior será 1.

In [249]:
tabla_1 = tabla
tabla_1.Presion = np.digitize(tabla.Presion, bins = [(48+60) / 2])
tabla_1

,Presion,Clase
0,0,+
1,0,+
2,1,-
3,1,-
4,1,-
5,1,+


In [250]:
G1 = H
for v in tabla_1.Presion.unique():
    #Mediante el metodo .loc, accedemos a las filas y columnas que indiquemos
    class1 = tabla_1.loc[tabla_1.Presion == v]
    G1 -= (len(class1)/len(tabla_1))*entropy(class1.Clase.value_counts(normalize=True), base=2)
    
print("Ganancia de información de la primera cota digitalización:", G1)

Ganancia de información de la primera cota digitalización: 0.4591479170272448


Ahora, calculamos la ganancia de información para la segunda cota de digitalización (Presión = 85).
Pero antes tenemos que volver a establecer los valores originales para la columna Presion de 'tabla'

In [251]:
tabla.Presion = [40, 48, 60, 72, 80, 90]

tabla_2 = tabla
tabla_2.Presion = np.digitize(tabla.Presion, bins = [(80+90) / 2])
tabla_2

,Presion,Clase
0,0,+
1,0,+
2,0,-
3,0,-
4,0,-
5,1,+


In [252]:
G2 = H
for v in tabla_2.Presion.unique():
    class2 = tabla_2.loc[tabla_2.Presion == v]
    G2 -= (len(class2)/len(tabla_2))*entropy(class2.Clase.value_counts(normalize=True), base=2)
    
print("Ganancia de información de la segunda cota digitalización:", G2)

Ganancia de información de la segunda cota digitalización: 0.19087450462110933


Como G1 > G2, se escogerá la primera cota de digitalización (Presión = 54)

In [253]:
tabla.Presion = [40, 48, 60, 72, 80, 90]

print("clf = DecisionTreeClassifier(max_leaf_nodes = 2, criterion = 'entropy')
clf.fit(np.array(tabla.Presion).reshape(-1, 1), tabla.Clase) #Mediante el método .reshape(-1,1) lo que hacemos es darle una nueva forma al array sin cambiar los datos, de manera que se muestre cada valor como una fila distinta

In [254]:
txt = export_text(clf, feature_names = ['Presión'])
print(txt)

|--- Presión <= 54.00
|   |--- class: +
|--- Presión >  54.00
|   |--- class: -

